In [ ]:
#input data file path
path = '...'
#input offsets that output from find_offsets
offsetx = -7
offsety = 3
#input frame squence
firstred = 10
green = 500
#other parameters
neighborhood_size = 5 #find local maxima within this range of radius
threshold = 200 #intensity threshold
edgecut = 9 #exlude pixels this far away from the edge in each channel
check_green = 10 #how many green frames used to find matching in D/A channels
min_distance = 5 #minimum distance for molecules considered as separate ones, i.e. not aggregating

In [14]:
from PIL import Image
import numpy as np
import scipy
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
import matplotlib.pyplot as plt

In [15]:
def read_tiff(path):
    img = Image.open(path)
    images = []
    for i in range(img.n_frames):
        img.seek(i)
        images.append(np.array(img))
        data = np.array(images)
    return data

In [16]:
data = read_tiff('PSD78/encapPSD78lp104.tif')

In [17]:
#define molecule extraction function for data file
def locate_molecules(offsetx, offsety, firstred, green, neighborhood_size, threshold, edgecut, check_green, min_distance):   
    #getting molecule positions map frame by frame
    maxima = []
    maximamap = np.zeros((560,512,512)) #local brightest pixel map for each frames
    for i in range(data.shape[0]):
        data_max = filters.maximum_filter(data[i,:,:], neighborhood_size) #local maximum filter
        maxima = (data[i,:,:] == data_max) #extract local brightest pixels w/ True to be the local brightest pixel
        data_min = filters.minimum_filter(data[i,:,:], neighborhood_size) #local minimum filter
        diff = ((data_max - data_min) > threshold) #extract pixels w/ intensity above threshold
        maxima[diff == 0] = 0
        maximamap[i,:,:] = maxima
    first_acceptormap = maximamap[0:firstred,:,256:512]
    fret_donormap = maximamap[firstred:green+firstred,:,0:256]
    fret_acceptormap = maximamap[firstred:green+firstred,:,256:512] #split D/A sides to make D/A map 
    
    #cut the edge by "edgecut" pixels and match D/A channel
    fret_molecules = np.zeros((check_green, 512, 256)) 
    for i in range(fret_molecules.shape[0]): #based on the A channel to match molecules
        for j in range(fret_molecules.shape[1]-2*edgecut):
            for k in range(fret_molecules.shape[2]-2*edgecut):
                if (fret_acceptormap[i,j+edgecut,k+edgecut] == 1) and \
                (fret_donormap[i,j+edgecut+offsety,k+edgecut+offsetx] == fret_acceptormap[i,j+edgecut,k+edgecut]):
                    fret_molecules[i,j+edgecut,k+edgecut] = 1 #apply the offsets that D is from A
    fret_molecules_map = (np.mean(fret_molecules, axis=0)!=0) #with value not equal to zero means at least match in one frame, cords are in A channel 
    molecule_cords = np.argwhere(fret_molecules_map == 1) #get final match molecules' cords in A channel
    #exclude aggregations
    y_cord=molecule_cords[:,0] 
    x_cord=molecule_cords[:,1] 
    xx=np.tile(x_cord,(len(x_cord),1)) #duplicate x cords by molecule number of rows
    yy=np.tile(y_cord,(len(y_cord),1)) #duplicate y cords by molecule number of rows
    xx_t=np.transpose(xx) #duplicate x cords by the molecule number of columns
    yy_t=np.transpose(yy) #duplicate y cords by the molecule number of columns
    distance=(yy-yy_t)**2+(xx-xx_t)**2 #distance^2 array, symmetric, dignal elements are 0
    overlap=np.argwhere((distance<min_distance**2) & (distance>0)).flatten() #compare with min_distance^2, smaller means aggregation; get aggregation column/row indices 
    overlap_n=np.array(list(set(overlap))) #finalize the aggregation molecules column/row indices(numbers)
    final_cords=np.delete(molecule_cords,overlap_n,0) #delete aggregation molecules 
    
    return [print('found this many molecules:' + str(final_cords.shape[0])), final_cords]
    

In [22]:
final_cords=locate_molecules(-7,3,10,500,3,200,9,10,5)[1]

found this many molecules:248


In [19]:
final_cords.shape

(190, 2)

# Faster Way!

In [1]:
#define molecule extraction function for data file
def locate_molecules(offsetx, offsety, firstred, green, neighborhood_size, threshold, edgecut, check_green, min_distance):
    #getting molecule positions map frame by frame
    maxima = []
    maximamap = np.zeros((560,512,512)) #local brightest pixel map for each frames
    for i in range(data.shape[0]):
        data_max = filters.maximum_filter(data[i,:,:], neighborhood_size) #local maximum filter
        maxima = (data[i,:,:] == data_max) #extract local brightest pixels w/ True to be the local brightest pixel
        data_min = filters.minimum_filter(data[i,:,:], neighborhood_size) #local minimum filter
        diff = ((data_max - data_min) > threshold) #extract pixels w/ intensity above threshold
        maxima[diff == 0] = 0
        maximamap[i,:,:] = maxima
    first_acceptormap = maximamap[0:firstred,:,256:512]
    fret_donormap = maximamap[firstred:green+firstred,:,0:256]
    fret_acceptormap = maximamap[firstred:green+firstred,:,256:512] #split D/A sides to make D/A map 
    
    #cut the edge by "edgecut" pixels and match D/A channel
    fret_molecules_cut = ((fret_acceptormap[0:check_green,edgecut:(fret_acceptormap.shape[1]-edgecut),edgecut:(fret_acceptormap.shape[2]-edgecut)] == 1)\
                          & \
                         (fret_acceptormap[0:check_green,edgecut:(fret_acceptormap.shape[1]-edgecut),edgecut:(fret_acceptormap.shape[2]-edgecut)] == \
                         fret_donormap[0:check_green,(edgecut-offsety):(fret_donormap.shape[1]-edgecut-offsety),(edgecut-offsetx):(fret_donormap.shape[2]-edgecut-offsetx)]))
    fret_molecules = np.zeros((check_green,512,256)) #detected matching molecules map for each frame
    fret_molecules[:,edgecut:(512-edgecut),edgecut:(256-edgecut)] = fret_molecules_cut
    fret_molecules_map = (np.mean(fret_molecules, axis=0)!=0) #with value not equal to zero means at least match in one frame, cords are in A channel 
    
    #exclude aggregations
    molecule_coords = np.argwhere(fret_molecules_map == 1) #extract matching molecules coords
    y_coord=molecule_coords[:,0] 
    x_coord=molecule_coords[:,1] 
    xx=np.tile(x_coord,(len(x_coord),1)) #duplicate x coords by molecule number of rows
    yy=np.tile(y_coord,(len(y_coord),1)) #duplicate y coords by molecule number of rows
    xx_t=np.transpose(xx) #duplicate x coords by the molecule number of columns
    yy_t=np.transpose(yy) #duplicate y coords by the molecule number of columns
    distance=(yy-yy_t)**2+(xx-xx_t)**2 #distance^2 array, symmetric, diagnal elements are 0
    overlap=np.argwhere((distance<min_distance**2) & (distance>0)).flatten() #compare with min_distance^2, smaller means aggregation; get aggregation column/row indices 
    overlap_n=np.array(list(set(overlap))) #finalize the aggregation molecules column/row indices(numbers)
    final_coords=np.delete(molecule_coords,overlap_n,0) #delete aggregation molecules 
    final_molecules_map = np.zeros((512,256)) #final map with aggregation removing
    final_molecules_map[final_coords[:,0],final_coords[:,1]] = 1
    
    
    return [print('found this many molecules:' + str(final_coords.shape[0])), final_molecules_map]
    